## HA2

Task1

In [ ]:
pip install bs4

In [28]:
import numpy as np
import pandas as pd
import requests
import json
from pprint import pprint
from bs4 import BeautifulSoup as bs

In [33]:
# ввод вакансии
#vacancy = input('Какую вакансию вы ищите: ')
vacancy = 'специалист по документообороту' #заглушка, чтобы не вбивать каждый раз текст
vacancy = ' '.join(vacancy.split())
vacancy = vacancy.replace(" ", "+")

# ввод количества страниц
#pages = int(input('Введите количество страниц: '))
pages = 5 #заглушка, чтобы не вбивать каждый раз текст
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36',
          'Accept' : '*/*'}

vacancy_name = []
link = []
company = []
city = []
pay = []
pay_min = []
pay_max = []
currency = []

for page in range(pages):
    
    url = f'https://hh.ru/search/vacancy?clusters=true&ored_clusters=true&enable_snippets=true&salary=&st=searchVacancy&text={vacancy}&page={page}'
    response = requests.get(url, headers = headers)
    
    if response.ok:
    
        soup = bs(response.text, 'html.parser')
        names = soup.find_all('div', attrs = {'class': ['vacancy-serp-item vacancy-serp-item_premium', 'vacancy-serp-item']})
        
        for name in [name for name in names if len(names) > 0]:
            
            # получаем название вакансию и ссылку на нее
            tag = 'data-qa'
            target = 'vacancy-serp__vacancy-title'         
            vacancy_name.append(name.find('a', attrs = {tag: target}).getText())
            link.append('https://hh.ru' + name.find('a', attrs = {tag: target})['href'].split("hh.ru", 1)[1].split("?from=vacancy_search_list", 1)[0]) 

            # получаем название компании
            tag = 'data-qa'
            target = 'vacancy-serp__vacancy-employer'        
            company.append(name.find('a', attrs = {tag: target}).getText().replace("\xa0", " "))

            # получаем название города
            tag = 'data-qa'
            target = 'vacancy-serp__vacancy-address' 
            city.append(name.find('span', attrs = {tag: target}).find(text=True, recursive=False))
        
            # мин и макс зарплату
            tag = 'data-qa'
            target = 'vacancy-serp__vacancy-compensation'
            if name.find('span', attrs = {tag: target}) != None:
                pay_data = name.find('span', attrs = {tag: target}).getText().replace("\u202f", "").replace("\xa0", "")
                pay.append(pay_data)
                if pay_data.split()[0] == 'от':
                    pay_min.append(pay_data.split()[1])
                    pay_max.append(np.nan)
                    currency.append(pay_data.split()[2])
                elif pay_data.split()[0] == 'до':
                    pay_min.append(np.nan)
                    pay_max.append(pay_data.split()[1])
                    currency.append(pay_data.split()[2])
                else:
                    pay_min.append(pay_data.split()[0])
                    pay_max.append(pay_data.split()[2])
                    currency.append(pay_data.split()[3])            
            else:
                pay.append(np.nan)
                pay_min.append(np.nan)
                pay_max.append(np.nan)
                currency.append(np.nan)
                

In [34]:

df = pd.DataFrame(list(zip(vacancy_name,link, company, city, pay_min, pay_max, currency)), 
                  columns = ['Вакансия', 'Link', 'Компания', 'Город' , 'Мин ЗП' , 'Макс ЗП', 'Валюта'])

df.to_csv('df.csv', index=False, sep= ',',encoding='utf-8-sig')
df.head(10)

,Вакансия,Link,Компания,Город,Мин ЗП,Макс ЗП,Валюта
0,Специалист по документообороту,https://hh.ru/vacancy/47086525,Минский колледж предпринимательства,Минск,1000,NaN,бел.руб.
1,Специалист по документообороту/делопроизводитель,https://hh.ru/vacancy/47050522,ООО АвтоТахограф,Санкт-Петербург,35000,40000,руб.
2,Специалист по документообороту/ Оператор 1С,https://hh.ru/vacancy/47126150,Клавторг,Москва,65000,NaN,руб.
3,Ведущий специалист по ведению документооборота,https://hh.ru/vacancy/47099156,Pony Express,Москва,64500,64500,руб.
4,Специалист по документообороту (Транспортная л...,https://hh.ru/vacancy/47031694,ООО BTG Internationale Spedition,Москва,60000,80000,руб.
5,Секретарь-делопроизводитель,https://hh.ru/vacancy/47130721,Мегалит,Санкт-Петербург,60000,60000,руб.
6,Офис-менеджер (делопроизводитель),https://hh.ru/vacancy/47120423,АО Инжиниринговый Центр Технодинамика,Москва,70000,77000,руб.
7,Специалист по документообороту в отделе аренде,https://hh.ru/vacancy/47150940,ООО Агропарк Сервис,Самара,NaN,35000,руб.
8,Специалист канцелярии,https://hh.ru/vacancy/47125455,АО Газпром-Медиа,Москва,40000,45000,руб.
9,Специалист по документообороту/Администратор о...,https://hh.ru/vacancy/47018602,ООО ПОС МЕДИА,Москва,60000,80000,руб.


Task 2